## Namespace

In [1]:
%%writefile namespace.yaml
apiVersion: v1
kind: Namespace
metadata:
  name: airflow

Writing namespace.yaml


## Webserver secret key

In [2]:
import secrets
print(secrets.token_hex(16))

a7854b0369154109fb0c28ae6fee221e


In [3]:
%%writefile secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: airflow-secret
  namespace: airflow
data:
  webserver-secret-key: a7854b0369154109fb0c28ae6fee221e

Writing secret.yaml


## Persistent Volume

In [4]:
%%writefile pv_dags.yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: airflow-dags
  labels:
    type: local
spec:
  storageClassName: microk8s-hostpath
  accessModes: ["ReadWriteMany"]
  capacity:
    storage: 50Gi
  hostPath:
    type: DirectoryOrCreate
    path: "/home/enlinea/Airflow/dags"

Writing pv_dags.yaml


In [5]:
%%writefile pvc_dags.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: airflow-dags
  namespace: airflow
spec:
  storageClassName: microk8s-hostpath
  volumeName: airflow-dags
  accessModes: ["ReadWriteMany"]
  resources:
    requests:
      storage: 50Gi

Writing pvc_dags.yaml


## Images

In [6]:
%%writefile requirements.txt
apache-airflow-providers-microsoft-mssql
apache-airflow-providers-mysql
apache-airflow-providers-apache-beam
apache-airflow-providers-google

Writing requirements.txt


In [9]:
%%writefile Dockerfile
FROM apache/airflow:latest-python3.8

USER root

RUN apt-get update -yqq \
    && apt-get install -yqq --no-install-recommends \
        build-essential \
    && apt-get upgrade -yqq \
    && apt-get autoremove --purge -yqq \
    && apt-get autoclean -yqq \
    && rm -rf /var/lib/apt/lists

USER airflow
COPY requirements.txt .

RUN python3 -m pip install --upgrade pip \
    && python3 -m pip install --no-cache-dir -qUr requirements.txt \
    && pip check \
    && rm -rf requirements.txt

Overwriting Dockerfile


## Install

In [6]:
%%writefile override.yaml
executor: "CeleryKubernetesExecutor"

webserverSecretKeySecretName: "airflow-secret"

# webserver-service
webserver:
  service:
    type: NodePort
    ports:
    - name: airflow-ui
      port: 8080
      nodePort: 30003
    
# repository
images:
  airflow:
    repository: "darklemon/airflow"
    tag: "latest"
    pullPolicy: Always
    
# dags
dags:
  persistence:
    enabled: true
    existingClaim: airflow-dags

Overwriting override.yaml


In [7]:
%%writefile install.sh
#!/bin/bash

export REPOSITORY="darklemon"
export IMAGE="airflow"
export TAG="latest"

docker build . --no-cache \
-f Dockerfile \
-t $REPOSITORY/$IMAGE:$TAG

docker push $REPOSITORY/$IMAGE:$TAG

microk8s helm3 repo add apache-airflow https://airflow.apache.org
microk8s helm3 repo update
microk8s kubectl create -f namespace.yaml
microk8s kubectl create -f pv_dags.yaml
microk8s kubectl create -f pvc_dags.yaml
microk8s kubectl create -f secret.yaml

microk8s helm3 install airflow apache-airflow/airflow \
--namespace airflow \
-f override.yaml

Overwriting install.sh


## Uninstall

In [8]:
%%writefile uninstall.sh
#!/bin/bash

microk8s helm3 delete airflow \
--namespace airflow
microk8s kubectl delete -f namespace.yaml
microk8s kubectl delete -f pv_dags.yaml
microk8s helm3 repo rm apache-airflow

Overwriting uninstall.sh
